<h1 style="color:lightcoral"> Exceptions handling in Python 1 </h1>

In [31]:
import os
import re
import sys
import socket
import traceback
import threading

In [3]:
# Enable the faulthandler, to handle expections and retrieve the causes of diagnosing crashes or faults
import faulthandler; faulthandler.enable()

Set the maximum depth of recursion bypassing the the default depth limit used to prevent excessive memory usage, <br>
due to recursive function calls => [10 millions].    

Set the stack (memory used for function call management and local variables) size for Python threads [2^27 bytes => 128 MB].

In [4]:
sys.setrecursionlimit(10**7)
threading.stack_size(2**27)

0

<h3 style="color:lightcoral"> <u> Example #1 </u> </h3>

In [5]:
""" Custom exception to know the exact nature of the error and its exact origin.
Useful for nested calls in different modules.
"""
number_five = 5
try:
    q = number_five + "ciao"
    
except Exception as e:
    print("##################################################")
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    msg = "Error! An Exception of type {0} occurred.\nNamely the exception is: \n{1!r}"
    message1 = msg.format(type(e).__name__, e.args)
    print(message1)
    print("The detected bug is at line {} of file: {}".format(exc_tb.tb_lineno,fname))
    error_is = traceback.format_exc().splitlines()
    bb = ''.join(error_is[2].split())
    print("The culprit is: {}".format(bb))
    print("##################################################")

##################################################
Error! An Exception of type TypeError occurred.
Namely the exception is: 
("unsupported operand type(s) for +: 'int' and 'str'",)
The detected bug is at line 3 of file: 3373730586.py
The culprit is: q=number_five+"ecco"
##################################################


<h3 style="color:lightcoral"> <u> Example #2 </u> </h3>

In [30]:
""" Generic Exception to catch all exceptions """
try:
    with open("inexistent_file.txt", "r") as file:
        content = file.read()
except Exception as e:
    print(f"An error occurred while processing the file: {e}")

An error occurred while processing the file: [Errno 2] No such file or directory: 'inexistent_file.txt'


In [26]:
""" Generic Exception to just continue the execution in another branch.
regexp:
- r: "raw string" prefix as always to treat backslashes as literal characters 
- \d: digits (0-9)
- +: quantifier to indicate that one or more occurrences of the desired pattern are considered
"""
#customized = 'ask_quest_111'
customized = 'ask_quest'
try:	
    if 'ask_quest' in customized: 
        plus_one_game = 1
        one_quest_id = int(re.search(r'\d+', customized).group())
except Exception as e:
    one_quest_id = 99999
    plus_one_game = 0

print(one_quest_id)

99999


<h3 style="color:lightcoral"> <u> Example #3 </u> </h3>

In [7]:
""" ZeroDivisionError """
try:
    # Code that may raise multiple exceptions
    result = 10 / 0
except (ZeroDivisionError, TypeError) as e:
    print("Caught exception:", str(e))

Caught exception: division by zero


<h3 style="color:lightcoral"> <u> Example #4 </u> </h3>

In [29]:
# Raise different exceptions
try:
    #result = int(1.3)
    #result = int("this_is_a_string")
    #result = int("this_is_a_string")
    print("ok result is ", result)
except ValueError:
    print("Caught ValueError: Unable to convert to int.")
except ZeroDivisionError:
    print("Caught ZeroDivisionError: Cannot divide by zero.")
except Exception as e:
    print("Caught general exception:", str(e))

ok result is  1


<h3 style="color:lightcoral"> <u> Example #5 </u> </h3>

In [9]:
try:
    # Code that may raise an exception
    result = 10 / 0
except ZeroDivisionError:
    print("Caught ZeroDivisionError: Cannot divide by zero.")
finally:
    print("Finally block executed.")

Caught ZeroDivisionError: Cannot divide by zero.
Finally block executed.


<h3 style="color:lightcoral"> <u> Example #6 </u> </h3>

In [10]:
def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero.")
    return a / b

try:
    # Call the function that may raise an exception
    result = divide(10, 0)
except ValueError as e:
    print("Caught ValueError:", str(e))

Caught ValueError: Cannot divide by zero.


<h3 style="color:lightcoral"> <u> Example #7 </u> </h3>

In [80]:
def connect_to_server(server_ip, port, timeout=4, normal_way=True, tim=False):
    """ Attempt to establish a connection to a remote server using a socket.
    Just used as example to raise various exceptions. 

    Parameters:
        - IP address or hostname of the remote server [str]
        - port number to connect to on the remote server [int]

    Raises:
        ConnectionError: If the connection to the server cannot be established
        TimeoutError: If the connection to the server times out
        PermissionError: If connecting to a privileged port without sufficient permissions
        socket.error: If an unexpected socket-related error occurs

    Details: 
        # sock.settimeout(timeout) will raise TimeoutError before other specific errors (like PermissionError).
        Added to test purpose.

    Usage: connect_to_server("example.com", 80)
    """
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(timeout)
            sock.connect((server_ip, port))
            if not normal_way:
                raise ConnectionError("A deliberate connection error. Deliberate simulation")
            if tim:
                raise TimeoutError("A deliberate timeout error. for deliberate simulation")
    except ConnectionError as connerr:
        print(f"ConnectionError: Failed to connect to the server: {connerr}")
    except TimeoutError as timerr:
        print(f"TimeoutError: Connection to the server timed out: {timerr}")
    except PermissionError as pe:
        print(f"PermissionError: Insufficient permissions to connect to port {port}: {pe}")
    except socket.error as sockerr:
        print(f"SocketError: An unexpected socket error occurred: {sockerr}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        

In [78]:
# Working 
try:
    connect_to_server("example.com", 80)
except KeyboardInterrupt:
    print("Connection attempt interrupted by the user.")
except Exception as e:
    print(f"An error occurred: {e}")

In [74]:
connect_to_server("invalid_ip_address", 80)

An unexpected error occurred: [Errno -2] Name or service not known


In [81]:
connect_to_server("example.com", 9999)

TimeoutError: Connection to the server timed out: timed out


In [68]:
connect_to_server(11, 80)

An unexpected error occurred: str, bytes or bytearray expected, not int


In [69]:
connect_to_server("example.com", 80, normal_way=False)

ConnectionError: Failed to connect to the server: A deliberate connection error. Deliberate simulation


In [70]:
connect_to_server("example.com", 80, tim=True)

TimeoutError: Connection to the server timed out: A deliberate timeout error. for deliberate simulation


In [83]:
connect_to_server("example.com", 8080)

TimeoutError: Connection to the server timed out: timed out
